In [69]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
customers_df = pd.read_csv(r"C:\Users\THAKALI\Downloads\Customers (2).csv")
products_df = pd.read_csv(r"C:\Users\THAKALI\Downloads\Products (2).csv")
transactions_df = pd.read_csv(r"C:\Users\THAKALI\Downloads\Transactions (2).csv")

In [71]:
merged_df = pd.merge(transactions_df, customers_df, on='CustomerID')
merged_df = pd.merge(merged_df, products_df, on='ProductID')

In [72]:
customer_features = merged_df.groupby('CustomerID').agg(
    total_purchase_value=('TotalValue', 'sum'),
    avg_transaction_value=('TotalValue', 'mean'),
    num_transactions=('TransactionID', 'count'),
    product_categories=('Category', lambda x: x.nunique())
).reset_index()

In [73]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.drop(columns=['CustomerID']))

In [74]:
similarity_matrix = cosine_similarity(scaled_features)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

In [75]:
lookalike_results = []
for customer_id in customer_features['CustomerID'][:20]:
    top_similar_customers = similarity_df.loc[customer_id].sort_values(ascending=False)[1:4]
    recommendations = [(cust_id, round(score, 2)) for cust_id, score in top_similar_customers.items()]
    lookalike_results.append({'CustomerID': customer_id, 'Recommendations': recommendations})


In [76]:
lookalike_df = pd.DataFrame(lookalike_results)
lookalike_df.to_csv('FirstName_LastName_Lookalike.csv', index=False)

In [77]:
print("Lookalike model results have been saved as FirstName_LastName_Lookalike.csv.")

Lookalike model results have been saved as FirstName_LastName_Lookalike.csv.
